In [4]:
pip install pymysql


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\송지훈\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
pip install selenium

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   ---------- ----------------------------- 2.4/9.4 MB 11.8 MB/s eta 0:00:01
   -------------------- ------------------- 4.7/9.4 MB 11.8 MB/s eta 0:00:01
   ------------------------------- -------- 7.3/9.4 MB 11.8 MB/s eta 0:00:01
   ---------------------------------------- 9.4/9.4 MB 11.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\송지훈\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pymysql
import time

# MySQL 연결 설정
conn = pymysql.connect(
    host='localhost',
    user='root',
    password='4205',
    database='qnet_crawling',
    charset='utf8mb4'
)
cursor = conn.cursor()

# DB 저장 함수
def save_to_db(first_category, second_category, items):
    # 1. 2차 분류 저장
    cursor.execute("INSERT INTO SecondCategory (name, first_category_name) VALUES (%s, %s)", (second_category, first_category))
    second_id = cursor.lastrowid

    # 2. 시행종목 저장
    for item in items:
        cursor.execute("INSERT INTO Item (name, second_category_id) VALUES (%s, %s)", (item, second_id))
    conn.commit()


# 크롬 드라이버 실행
driver = webdriver.Chrome()
driver.get("https://q-net.or.kr/crf005.do#none")
wait = WebDriverWait(driver, 10)

print("🔍 크롤링 시작...\n")

# 1차 직무 분야 수집
all_first_level_links = wait.until(EC.presence_of_all_elements_located(
    (By.CSS_SELECTOR, "a[data-target^='duty_step2_']")
))
first_level_links = [elem for elem in all_first_level_links if elem.is_displayed()]

for i in range(len(first_level_links)):
    # 새로고침 후 재수집
    all_first_level_links = wait.until(EC.presence_of_all_elements_located(
        (By.CSS_SELECTOR, "a[data-target^='duty_step2_']")
    ))
    first_level_links = [elem for elem in all_first_level_links if elem.is_displayed()]
    
    first = first_level_links[i]
    first_text = first.text.strip()
    print(f"▶ 1차 클릭: {first_text}")
    driver.execute_script("arguments[0].click();", first)
    time.sleep(0.5)

    # 2차 분류 수집
    try:
        all_second_level_links = wait.until(EC.presence_of_all_elements_located(
            (By.CSS_SELECTOR, "a[data-target^='duty_step3_']")
        ))
        second_level_links = [elem for elem in all_second_level_links if elem.is_displayed()]
    except:
        print("  ⚠ 2차 분류 없음")
        continue

    for j in range(len(second_level_links)):
        all_second_level_links = wait.until(EC.presence_of_all_elements_located(
            (By.CSS_SELECTOR, "a[data-target^='duty_step3_']")
        ))
        second_level_links = [elem for elem in all_second_level_links if elem.is_displayed()]

        second = second_level_links[j]
        second_text = second.text.strip()
        print(f"  ▶ 2차 클릭: {second_text}")
        driver.execute_script("arguments[0].click();", second)
        time.sleep(0.5)

        # 시행종목 수집
        try:
            third_links = WebDriverWait(driver, 5).until(
                EC.presence_of_all_elements_located(
                    (By.CSS_SELECTOR, "div.qualList.qulList_lv4 div.limitY div.item ul li a[onclick^='jmDetail']")
                )
            )
            visible_third_links = [link for link in third_links if link.is_displayed()]
            if visible_third_links:
                print(f"    ▶ 시행종목:")
                item_names = []
                for link in visible_third_links:
                    name = link.text.strip()
                    item_names.append(name)
                    print("      -", name)
                
                # ✅ DB에 저장
                save_to_db(first_text, second_text, item_names)
            else:
                print("    ⚠ 시행종목 없음")
        except:
            print("    ⚠ 시행종목 없음 또는 로딩 실패")

    # 페이지 새로고침
    driver.get("https://q-net.or.kr/crf005.do#none")
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a[data-target^='duty_step2_']")))
    time.sleep(0.5)

print("\n✅ 크롤링 완료!")
driver.quit()
conn.close()


🔍 크롤링 시작...

▶ 1차 클릭: 사업관리
  ▶ 2차 클릭: 사업관리
    ▶ 시행종목:
      - 공공조달관리사
▶ 1차 클릭: 경영.회계.사무
  ▶ 2차 클릭: 경영
    ▶ 시행종목:
      - 사회조사분석사1급
      - 사회조사분석사2급
      - 소비자전문상담사1급
      - 소비자전문상담사2급
      - 컨벤션기획사1급
      - 컨벤션기획사2급
  ▶ 2차 클릭: 경영(사회조사분석)
    ⚠ 시행종목 없음 또는 로딩 실패
  ▶ 2차 클릭: 경영(소비자전문상담)
    ⚠ 시행종목 없음 또는 로딩 실패
  ▶ 2차 클릭: 경영(컨벤션기획)
    ⚠ 시행종목 없음 또는 로딩 실패
  ▶ 2차 클릭: 회계
    ⚠ 시행종목 없음 또는 로딩 실패
  ▶ 2차 클릭: 사무
    ⚠ 시행종목 없음 또는 로딩 실패
  ▶ 2차 클릭: 생산관리
    ▶ 시행종목:
      - 공장관리기술사
      - 포장기사
      - 포장기술사
      - 포장산업기사
      - 품질경영기사
      - 품질경영산업기사
      - 품질관리기술사
▶ 1차 클릭: 금융.보험
  ▶ 2차 클릭: 금융.보험
    ⚠ 시행종목 없음 또는 로딩 실패
▶ 1차 클릭: 교육.자연.과학.사회과학
  ▶ 2차 클릭: 교육.자연.과학.사회과학
    ▶ 시행종목:
      - 이러닝운영관리사
▶ 1차 클릭: 법률.경찰.소방.교도.국방
  ▶ 2차 클릭: 법률.경찰.소방.교도.국방
    ⚠ 시행종목 없음 또는 로딩 실패
▶ 1차 클릭: 보건.의료
  ▶ 2차 클릭: 보건.의료
    ▶ 시행종목:
      - 국제의료관광코디네이터
      - 임상심리사1급
      - 임상심리사2급
  ▶ 2차 클릭: 보건.의료(국제의료관광코디네이터)
    ⚠ 시행종목 없음 또는 로딩 실패
▶ 1차 클릭: 사회복지.종교
  ▶ 2차 클릭: 사회복지.종교
    ▶ 시행종목:
      - 직업상담사1급
      - 직업상담사2급
